In [1]:
import pandas as pd
import numpy as np

raw_users = pd.read_json('data/users.jsonl', lines=True)
raw_products = pd.read_json('data/products.jsonl', lines=True)
raw_sessions = pd.read_json('data/sessions.jsonl', lines=True)
raw_deliveries = pd.read_json('data/deliveries.jsonl', lines=True)

In [2]:
raw_users.dtypes

user_id     int64
name       object
city       object
street     object
dtype: object

In [3]:
raw_products.dtypes

product_id         int64
product_name      object
category_path     object
price            float64
dtype: object

In [4]:
raw_deliveries.dtypes

purchase_id             int64
purchase_timestamp     object
delivery_timestamp     object
delivery_company      float64
dtype: object

In [5]:
raw_sessions.dtypes

session_id                   int64
timestamp           datetime64[ns]
user_id                    float64
product_id                 float64
event_type                  object
offered_discount             int64
purchase_id                float64
dtype: object

### Clean user data

In [6]:
users = raw_users.astype({
    'user_id': 'float64'
})
users.set_index('user_id')
users.dtypes

user_id    float64
name        object
city        object
street      object
dtype: object

In [7]:
users.head()

,user_id,name,city,street
0,102.0,Juliusz Męcik,Warszawa,plac Krakowska 69
1,103.0,Eliza Miętka,Radom,ul. Bursztynowa 56/10
2,104.0,Kacper Kolarz,Warszawa,plac Wiślana 99
3,105.0,Tobiasz Radko,Gdynia,ulica Zwycięstwa 76/98
4,106.0,Roksana Merchel,Kutno,plac Głogowa 53/95


### Clean product data

In [8]:
products = raw_products.astype({
    'product_id': 'float64', 
    'category_path': 'unicode',
})
products.set_index('product_id')
categories = products.category_path.apply(lambda s: s.split(';')).apply(pd.Series)
categories = categories.rename(columns = lambda x : 'cat_' + str(x))
products = pd.concat([products, categories], axis=1)
products['main_category'] = products.categories.apply(lambda s: s[0])
products.head()

AttributeError: 'DataFrame' object has no attribute 'categories'

In [ ]:
# analyse invalid products
invalid_prices_filter = (products['price']>0) & (products['price']<100000)

pd.concat([
    products['main_category'].value_counts().rename('Products'), 
    products[~invalid_prices_filter]['main_category'].value_counts().rename('Invalid products')
          ], axis=1).fillna(value=0).sort_values(by='Products', ascending=False).plot(kind='bar', 
        title='Invalid products by main category')

cp_df = pd.concat([
    products['category_path'].value_counts().rename('Products'), 
    products[~invalid_prices_filter]['category_path'].value_counts().rename('Invalid products')
          ], axis=1).fillna(value=0).sort_values(by='Products', ascending=False)
cp_df[cp_df['Invalid products']>0].plot(kind='bar', title='Invalid products by category path')

In [ ]:
cat_df = pd.concat([
    products['category_path'].value_counts().rename('Products'), 
    products[~invalid_prices_filter]['category_path'].value_counts().rename('Invalid products')
          ], axis=1).fillna(value=0)
cat_df['invalid_percent'] = cat_df.apply(lambda r: round(100*r['Invalid products']/r['Products'],1), axis=1)
cat_df[cat_df['invalid_percent'] > 0].sort_values(by='Invalid products', ascending=False)


In [ ]:
# Filter invalid prices: out of range (0.01, 100 000)
products = products[invalid_prices_filter]
print('Removed {:.1f}% of rows (before {}, after {}).'.format(
    100-100*len(products)/len(raw_products), len(products), len(raw_products)))

In [ ]:
products.head()

### Clean session data

In [ ]:
sessions = raw_sessions.astype({
    'session_id': 'float64'
})
sessions['made_purchase'] = sessions.purchase_id.map(lambda x: False if pd.isnull(x) else True)

sessions.dtypes

#### Invalid sessions OVER CONVERSIONS

In [ ]:
invalid_session_cond = sessions['product_id'].isin(products['product_id'].unique())
sessions['is_invalid'] = sessions['product_id'].isin(products['product_id'].unique())

sessions.groupby(['is_invalid', 'made_purchase']).session_id.count()

#### Invalid sessions OVER USERS

In [ ]:
plt = pd.concat([
    sessions['user_id'].value_counts().rename('User sessions'), 
    sessions[~invalid_session_cond]['user_id'].value_counts().rename('User invalid sessions')
          ], axis=1).fillna(value=0).sort_values(by='User sessions', ascending=False)\
    .plot(kind='bar', title='Invalid sessions over users', figsize=(15,5), xticks=None).axis('off')


df = pd.concat([
    sessions['user_id'].value_counts(dropna=False).rename('User sessions'), 
    sessions[~invalid_session_cond]['user_id'].value_counts(dropna=False).rename('User invalid sessions')
          ], axis=1).fillna(value=0).sort_values(by='User sessions', ascending=False)
df['invalid_percent_per_user'] = df.apply(lambda r: 100*r['User invalid sessions']/r['User sessions'], axis=1)
df['invalid_percent_per_user'].describe()

#### Invalid sessions OVER TIME

In [ ]:
sessions['timestamp_date'] = sessions['timestamp'].dt.date

pd.concat([
    sessions['timestamp_date'].value_counts().rename('Sessions'), 
    sessions[~invalid_session_cond]['timestamp_date'].value_counts().rename('Invalid sessions')
          ], axis=1).fillna(value=0).sort_values(by='Sessions', ascending=True)\
    .plot(kind='bar', title='Invalid sessions over timestamp.date').axis('off')


#### Filtering sessions

In [ ]:
sessions = sessions[invalid_session_cond]
print('Removed {:.1f}% of rows (before {}, after {}).'.format(
    100-100*len(sessions)/len(raw_sessions), len(sessions), len(raw_sessions)))


sessions.head()

### Clean delivery data

In [ ]:
deliveries = raw_deliveries.astype({
    'purchase_id': 'float64', 
    'delivery_timestamp': 'datetime64',
    'purchase_timestamp': 'datetime64'
})
deliveries.set_index('purchase_id')

deliveries.dtypes

In [ ]:
deliveries['delivery_time'] = deliveries.apply(lambda row: row['delivery_timestamp'] - row['purchase_timestamp'], axis=1)
deliveries.head()

In [ ]:
deliveries = deliveries[deliveries.purchase_id.isin(sessions['purchase_id'])]
print('Removed {:.1f}% of rows (before {}, after {}).'.format(
    100-100*len(deliveries)/len(raw_deliveries), len(deliveries), len(raw_deliveries)))


## Merge datasets

In [ ]:
data = sessions.merge(products, how='left', on='product_id')
data = data.merge(users, how='left', on='user_id')
data = data.merge(deliveries, how='left', on='purchase_id')
data = data.drop(['category_path'], axis=1)
data.head()

In [ ]:
data.to_json('data/merged.jsonl', orient='records', lines=True)